In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets,models,transforms
import time
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [2]:
data_transforms={
    'train':transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])        
    ]),
    'val':transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
}

data_dir='D:/数据库/dog_vs_cat/'
class_numbers=len(os.listdir(data_dir+'train'))

image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}
dataloaders={x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True) for x in ['train','val']}
dataset_sizes={x:len(image_datasets[x]) for x in ['train','val']}

In [3]:
model_ft=models.densenet121(pretrained=True)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\user/.torch\models\densenet121-a639ec97.pth
2293760it [08:09, 4681.32it/s] 


ProtocolError: ("Connection broken: ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)", ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

In [ ]:
num_ftrs=model_ft.classifier.in_features
model_ft.classifier=nn.Linear(num_ftrs,class_numbers)

model_ft=model_ft.cuda()

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer_ft=optim.SGD(model_ft.parameters(),lr=0.001,momentum=0.9)
exp_lr_scheduler=lr_scheduler.StepLR(optimizer_ft,step_size=7,gamma=0.1)

In [ ]:
since=time.time()
beat_model=model_ft.state_dict()
bast_acc=0.0

num_epochs=25
for epoch in range(num_epochs):
    print(f'Epoch {epoch}-{num_epochs-1}')
    print('-'*30)
    
    for phase in ['train','val']:
        if phase == 'train':
            exp_lr_scheduler.step()
            model_ft.train(True)
        else:
            model_ft.train(False)
            
        running_loss=0.0
        running_corrects=0
        
        for data in dataloaders[phase]:
            inputs,labels=data
            inputs=Variable(inputs)
            labels=Variable(labels)     
            
            optimizer_ft.zero_grad()
            
            outputs=model_ft(inputs)
            _,preds=torch.max(outputs.data,1)
            loss=criterion(outputs,labels)
            
            if phase == 'train':
                loss.backward()
                optimizer.step()
                
            running_loss+=loss.item()
            running_corrects+=torch.sum(preds==labels.data)
            
        epoch_loss=running_loss/datasets[phase]
        epoch_acc=running_corrects/datasets[phase]   
        
        print(f'{phase} loss: {epoch_loss} acc: {epoch_acc}')
        
        if phase=='val' and epoch_acc>best_acc:
            best_acc=epoch_acc
            best_model=model_ft.state_dict()